In [1]:
import pandas as pd 
import json

In [2]:
import json

# Load JSON data from a file
with open('assets/my_topics.json') as file:
    json_data = json.load(file)

# Convert JSON data to a dictionary
my_topics = dict(json_data)

# Load JSON data from a file
with open('assets/topics.json') as file:
    json_data = json.load(file)

# Convert JSON data to a dictionary
topics = dict(json_data)

In [1]:
from sklearn.metrics import jaccard_score

def calculate_similarity(topics1, topics2):
    set1 = set(topics1)
    set2 = set(topics2)
    similarity = jaccard_score(set1, set2)
    return similarity

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Calculate similarity between books in books1 and books2
similarity_matrix = []
for title1, topics1 in books1.items():
    similarities = []
    for title2, topics2 in books2.items():
        similarity = calculate_similarity(topics1, topics2)
        similarities.append(similarity)
    similarity_matrix.append(similarities)

# Create a DataFrame from the similarity matrix
book1_titles = list(books1.keys())
book2_titles = list(books2.keys())
similarity_df = pd.DataFrame(similarity_matrix, columns=book2_titles, index=book1_titles)

print(similarity_df)